In [ ]:
import pandas as pd
import os
from exts_used_count import *
import regex as re

In [ ]:
db_path = '../../datasets/'
json_path = '../../json_datasets'

In [ ]:
# dictionary which counts how many times the same column name is used in all datasets
columnName2datasets = {}
# dictionary which counts how many times x (i.e. a number) columns are in all datasets
numColumns2datasets = {}
# dictionary which counts how many times x (i.e. a number) rows are in all datasets
numRows2datasets = {}
# tot rows in all datasets
tot_num_row = 0
# tot cols in all datasets
tot_num_col = 0

for file in os.listdir(json_path):
    filepath = f"{json_path}/{file}"
    dataset = pd.read_json(filepath)
    # get columns name (case insensitive)
    columnsName = dataset.axes[1]
    # get number of rows of current dataset
    num_row_dataset = len(dataset.axes[0])
    # get number of columns of current dataset
    num_column_dataset = len(columnsName)
    # update total number of rows of all datasets
    tot_num_row += num_row_dataset
    # update total number of columns of all datasets
    tot_num_col += num_column_dataset

    # update dictionaries
    if num_row_dataset not in numRows2datasets:
        numRows2datasets[num_row_dataset] = 1
    else:
        numRows2datasets[num_row_dataset] += 1

    if num_column_dataset not in numColumns2datasets:
        numColumns2datasets[num_column_dataset] = 1
    else:
        numColumns2datasets[num_column_dataset] += 1
    
    for colName in columnsName:
        # Le colonne lette messe in lower case
        colName = colName.lower()
        # colName = re.sub("_"," ", colName)
        if colName in columnName2datasets:
            columnName2datasets[colName] += 1
        else:
            columnName2datasets[colName] = 1

print("Tot cols:", tot_num_col)
print("Tot rows:", tot_num_row)


## Ordered by key

In [ ]:
import collections
# Ordino le mappe per chiavi
numColumns2datasetsByKey = collections.OrderedDict(sorted(numColumns2datasets.items()))
numRows2datasetsByKey = collections.OrderedDict(sorted(numRows2datasets.items()))
columnName2datasetsByKey = collections.OrderedDict(sorted(columnName2datasets.items()))

In [ ]:
print("columnName2datasets", columnName2datasetsByKey)

In [ ]:
print("numColumns2datasets", numColumns2datasetsByKey)

In [ ]:
print("Rows2datasets", numRows2datasetsByKey)

## Ordered by value

In [ ]:
# Ordina le mappe per valore
numColumns2datasetsByValue = {numCol: numDb for numCol, numDb in sorted(numColumns2datasets.items(), key=lambda item: item[1], reverse=True)}
numRows2datasetsByValue = {numRow: numDb for numRow, numDb in sorted(numRows2datasets.items(), key=lambda item: item[1], reverse=True)}
columnName2datasetsByValue = {nameCol: numDb for nameCol, numDb in sorted(columnName2datasets.items(), key=lambda item: item[1], reverse=True)}

In [ ]:
print("columnName2datasets", columnName2datasetsByValue)

In [ ]:
pd.DataFrame({key : [columnName2datasetsByValue[key]] for key in columnName2datasetsByValue})

In [ ]:
print("numColumns2datasets", numColumns2datasetsByValue)

In [ ]:
print("Rows2datasets", numRows2datasetsByValue)

In [ ]:
print("File types used", exts_count(db_path))

In [ ]:
# Funzione che calcola la distribuzione di 
# 'num2count' su 'steps' intervalli
def compute_distribution(num2count, steps = 5):
    nums = num2count.keys()
    
    # calcol minimo e massimo valore 
    # delle chiavi di 'num2count'
    minNum = min(nums)
    maxNum = max(nums)

    # Questo (steps + 1) aggiusta alcuni problemi
    # su Lower e Upper bound ~non so il perchè~
    # (ho pensato che 'lb + 1' incrementi lo 'stepSize'
    # di '1' ad ogni fine step per un incremento totale
    # di 'steps - 1' sui bound)
    length = (maxNum - minNum) - steps + 1 
    
    if length <= 0: return  
    
    stepSize = length // steps
    r = length % steps
    
    lb = minNum
    step2count = {}
    for _ in range(steps):
        ub = lb + stepSize + int(r > 0)

        step2count[(lb, ub)] = 0
        for num in nums:
            if (lb <= num and num <= ub):
                step2count[(lb, ub)] += num2count[num]
        lb = ub + 1
        r -= 1

    return step2count


In [ ]:
df = compute_distribution(numColumns2datasets, 5)
pd.DataFrame({ key: [df[key]] for key in df})

In [ ]:
df = compute_distribution(numRows2datasets, 5)
pd.DataFrame({ key: [df[key]] for key in df})

# Informazioni catalogate per anni

In [ ]:
import regex as re
l = list(columnName2datasetsByValue.keys())
r = re.compile('.*[0-9].*')
ret = list(filter(r.match,l))
for i in ret:
    if "revenue" in i:
        print(i)

for i in ret:
    if "employees" in i:
        print(i)